In [5]:
# VAEエンコーダネットワーク
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import numpy as np

img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2  # Dimensionality of the latent space: a plane

input_img = tensorflow.keras.Input(shape=img_shape)

x = layers.Conv2D(32, 3,
                  padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3,
                  padding='same', activation='relu',
                  strides=(2, 2))(x)
x = layers.Conv2D(64, 3,
                  padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3,
                  padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [6]:
# 潜在空間サンプリング関数
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

In [7]:
# 潜在空間の点を画像マッピングするVAEデコーダネットワーク
# この入力でzを供給
decoder_input = layers.Input(K.int_shape(z)[1:])

# 入力を正しい数のユニットにアップサンプリング
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)

# 最後のFlatten層の直前の特徴マップと同じ形状の特徴マップに変換
x = layers.Reshape(shape_before_flattening[1:])(x)

# Conv2DTranspose層とCon２D層を使って，元の入力画像と同じサイズの特徴マップに変換
x = layers.Conv2DTranspose(32, 3,
                           padding='same', activation='relu',
                           strides=(2, 2))(x)
x = layers.Conv2D(1, 3,
                  padding='same', activation='sigmoid')(x)

# decoder_inputをデコードされた画像に変換するデコーダモデルをインスタンス化
decoder = Model(decoder_input, x)

# このモデルzに適用デコードされたzを復元
z_decoded = decoder(z)

In [8]:
# VAEの損失関数を計算するためのカスタム層
class CustomVariationalLayer(tensorflow.keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = tensorflow.keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        # この出力は使用しないが，層は何か返さなければならない
        return x

# カスタム層を呼び出し，最終的なモデル出力を取得するための入力とデコードされた出力渡す．
y = CustomVariationalLayer()([input_img, z_decoded])

In [13]:
from tensorflow.keras.datasets import mnist
tensorflow.config.experimental_run_functions_eagerly(True)

vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

# Mnistの手書きの数字でVAEを訓練
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

vae.fit(x=x_train, y=None,
        shuffle=True,
        epochs=10,
        batch_size=batch_size,
        validation_data=(x_test, None))


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 14, 14, 64)   36928       conv2d_6[0][0]                   
____________________________________________________________________________________________

NotImplementedError: Cannot convert a symbolic Tensor (truediv_99:0) to a numpy array.